# 부동산 정보 크롤링 
## 데이터 출처 조인스 - 서비스 중지

http://price.joinsland.joins.com/

http://price.joinsland.joins.com/area/index.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=1168010600

다음 URL의 인자로 다음 값들이 전달.

http://price.joinsland.joins.com/ajax/price.info.dongapt.asp
* mcateGroup: A1=아파트, A6=오피스텔
* mcateCode: A1A3A4
* areaCode=1168010600

결과적으로 지역코드(areaCode)만 지정하면 가격  데이터를 가져올 수 있다.

# 지역이름과 코드
(참고) joinsland.joins.com 자체에도 법정동을 조회하는 내용이 있으나 데이터가 온전한 JSON 이 아니어서 가공하기 어렵다

In [1]:
# 구/시/군 (MCODE) 코드 가져오기

import requests

# 서울특별시 하위 '구/시/군' 읽기
area_code = '110000000' 
url = 'http://price.joinsland.joins.com/ajax/area_search.asp?div=MCODE&areaCode=' + area_code

r = requests.get(url)
print(r.text)

# JSON처럼 보이지만 JSON 규격을 따르지 않고 있다

<!DOCTYPE html>
<html lang="ko">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta http-equiv="Content-Script-Type" content="text/javascript">
<meta http-equiv="Content-Style-Type" content="text/css">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=1060">
<meta name="robots" content="index">
<meta property="og:type" content="website"/>
<meta property="og:title" content="ì¡°ì¸ì¤ëë ë¶ëì° ìë¹ì¤ ì¢ë£"/>
<meta property="og:url" content="https://joinsland.joins.com/"/>
<title>ì¡°ì¸ì¤ëë ë¶ëì° ìë¹ì¤ ì¢ë£</title>
</head>
<body>
<table style="width:100%;">
<tr>
	<td style="text-align:center;padding-top:100px"><img src="end_joinsland.png" style="width:616px;height:426px" alt="ìë¹ì¤ê° ì¢ë£ ëììµëë¤. ì¡°ì¸ì¤ëë ë¶ëì° ìë¹ì¤ë¥¼ ë¹ì¬ì ì¬ìê°í¸ì¼ë¡ 2023ë 7ì 31ì¼(ì)ìë¡ ì¢ë£íê² ëììµëë¤. ê·¸ëì ì¡°ì¸ì¤ëë ì¬ì´í¸ë¥¼ ì´ì©í´ ì£¼ì 

# 법정동 이름과 코드
부동산 정보등에는 법정동 코드가 사용된다. 
* 법정동 코드(8자리)
* 2(시/도) + 2자리(구/군/구) + 2자리(읍/면/동) + 2자리(리/단지)

법정동 전체 코드는 아래 링크

* https://goo.gl/tM6r3v

상세한 내용은 다음 링크에 설명을 참고

* http://nbviewer.jupyter.org/urls/financedata.github.io/posts/korea-area-code.ipynb

In [2]:
import pandas as pd

def get_areacode():
    df_areacode = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str, '법정동명':str})
    df_areacode = df_areacode[df_areacode['폐지여부'] == '존재']
    df_areacode = df_areacode[ ['법정동코드', '법정동명'] ]
    return df_areacode

def get_province():
    df_areacode = get_areacode()
    df_province = df_areacode[ df_areacode['법정동코드'].str.contains('\d{2}0{8}|36110{6}')]
    return df_province

In [3]:
df_areacode = get_areacode()
df_province = get_province()

df_province

,법정동코드,법정동명
0,1100000000,서울특별시
2615,2600000000,부산광역시
2912,2700000000,대구광역시
3244,2800000000,인천광역시
3579,2900000000,광주광역시
3825,3000000000,대전광역시
4009,3100000000,울산광역시
4241,3611000000,세종특별자치시
4391,4100000000,경기도
10867,4200000000,강원도


In [4]:
df_areacode.head(3)

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동


In [ ]:
df_areacode[ df_areacode['법정동명'].str.contains('서울특별시 강남구') ]

# 데이터 전처리

http://price.joinsland.joins.com/ajax/price.info.dongapt.asp
* mcateGroup A1=아파트, A6=오피스텔
* mcateCode: A1A3A4
* areaCode=1168010600 # 강남구 대치동

In [ ]:
import pandas as pd
url = 'http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=' + '1168010600'
    
print(url)
dfs = pd.read_html(url,encoding='utf-8')
df = dfs

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=' + '1168010600'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

In [ ]:
table = soup.find('table')
trs = table.tbody.find_all('tr')

In [ ]:
rowspan_val = ''

for tr in trs[:20]: # 처음 20개 확인
    tds = tr.find_all('td')
    if tds[0].has_attr('rowspan'):
        rowspan_val = tds[0].text
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text, tds[5].text
    else:
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = rowspan_val, tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text
    매매가_저 = int(매매가.split(' ~ ')[0].replace(',', ''))
    매매가_고 = int(매매가.split(' ~ ')[1].replace(',', ''))
    전세가_저 = int(전세가.split(' ~ ')[0].replace(',', ''))
    전세가_고 = int(전세가.split(' ~ ')[1].replace(',', ''))
    print(단지명, 면적, 매매가_저, 매매가_고, 매물, 전세가_저, 전세가_고, 전세_매물)

In [ ]:
# DataFrame으로 만들기
values_list = []

table = soup.find('table', attrs={'class':'tbl_compare'})
trs = table.tbody.find_all('tr')

rowspan_val = ''

for tr in trs:
    tds = tr.find_all('td')
    if tds[0].has_attr('rowspan'):
        rowspan_val = tds[0].text
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text, tds[5].text
    else:
        단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = rowspan_val, tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text
    매매가_저 = int(매매가.split(' ~ ')[0].replace(',', ''))
    매매가_고 = int(매매가.split(' ~ ')[1].replace(',', ''))
    전세가_저 = int(전세가.split(' ~ ')[0].replace(',', ''))
    전세가_고 = int(전세가.split(' ~ ')[1].replace(',', ''))
    values_list.append([단지명, 면적, 매매가_저, 매매가_고, 매물, 전세가_저, 전세가_고, 전세_매물])
    
cols = ['단지명', '면적', '매매가_저', '매매가_고', '매물', '전세가_저', '전세가_고', '전세가_매물']   
df = pd.DataFrame(values_list, columns=cols)

print ("건수:", len(df))
df.head(3)

# 함수로 만들기

In [ ]:
# 조인스부동산 아파트 조회

import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_areacode():
    df_areacode = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str, '법정동명':str})
    df_areacode = df_areacode[df_areacode['폐지여부'] == '존재']
    df_areacode = df_areacode[['법정동코드', '법정동명']]
    return df_areacode

def get_province():
    df_areacode = get_areacode()
    df_province = df_areacode[ df_areacode['법정동코드'].str.contains('\d{2}0{8}|36110{6}')]
    return df_province


def joins_realasset(areacode = ''):
    url = 'http://price.joinsland.joins.com/ajax/price.info.dongapt.asp?mcateGroup=A1&mcateCode=A1A3A4&areaCode=' + areacode

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    values_list = []

    table = soup.find('table', attrs={'class':'tbl_compare'})
    trs = table.tbody.find_all('tr')
    rowspan_val = ''

    for tr in trs:
        tds = tr.find_all('td')
        if tds[0].has_attr('rowspan'):
            rowspan_val = tds[0].text
            단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text, tds[5].text
        else:
            단지명, 면적, 매매가, 매물, 전세가, 전세_매물 = rowspan_val, tds[0].text, tds[1].text, tds[2].text, tds[3].text, tds[4].text
        매매가_저 = int(매매가.split(' ~ ')[0].replace(',', ''))
        매매가_고 = int(매매가.split(' ~ ')[1].replace(',', ''))
        전세가_저 = int(전세가.split(' ~ ')[0].replace(',', ''))
        전세가_고 = int(전세가.split(' ~ ')[1].replace(',', ''))
        values_list.append([단지명, 면적, 매매가_저, 매매가_고, 매물, 전세가_저, 전세가_고, 전세_매물])

    cols = ['단지명', '면적', '매매가_저', '매매가_고', '매물', '전세가_저', '전세가_고', '전세가_매물']   
    df = pd.DataFrame(values_list, columns=cols)
    return df

# 검색과 활용

In [ ]:
df_areacode = get_areacode()

In [ ]:
area = '신사동'
area_code = df_areacode[ df_areacode['법정동명'].str.contains(area) ]
area_code

In [ ]:
area = '강남구 신사동'
area_code = df_areacode[ df_areacode['법정동명'].str.contains(area) ]['법정동코드'].values[0]
area_code

In [ ]:
joins_realasset(area_code)

----
### 2017 FinanceData http://financedata.github.com , http://fb.com/financedata